In [1]:
%pip install dowhy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.5/989.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# imports
import networkx as nx 
import numpy as np
import pandas as pd
from dowhy import gcm

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# loading the data 
depression = pd.read_csv("data/Student Depression Dataset.csv")
depression = depression.dropna()
depression = depression.replace({'Yes': 1, 'No': 0})

depression_LASSO_features = depression[['Academic Pressure', 'Have you ever had suicidal thoughts ?',
 'Financial Stress', 'City_Ahmedabad', 'City_Bhopal', 'City_Faridabad',
 'City_Hyderabad', 'City_Meerut', 'City_Patna', 'Dietary Habits_Moderate',
 'Dietary Habits_Others', 'Dietary Habits_Unhealthy', 'Depression']]

depression_LASSO_features.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/Student Depression Dataset.csv'

In [ ]:
# X -> treatment (e.g., sleep duration, dietary habits)
# Y -> city of residence?
# Z -> observation (e.g., depression)

# create "scoring" for dietary habits -> 0 for unhealthy, 1 for moderate, 2 for other 

X = depression_LASSO_features['Dietary Habits_Unhealthy'].replace({})

X = depression_LASSO_features['']
Y = ...
Z = ...

In [ ]:
training_data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z))

In [ ]:
causal_model = gcm.InvertibleStructuralCausalModel(nx.DiGraph([('X', 'Y'), ('Y', 'Z')])) # X -> Y -> Z
causal_model.set_causal_mechanism('X', gcm.EmpiricalDistribution())
causal_model.set_causal_mechanism('Y', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
causal_model.set_causal_mechanism('Z', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

In [ ]:
gcm.fit(causal_model, training_data)

In [ ]:
gcm.counterfactual_samples(
    causal_model,
    {'X': lambda x: 2},
    observed_data=pd.DataFrame(data=dict(X=[1], Y=[2], Z=[3])))